In [2]:
import numpy as np
import json
import pandas as pd

### 链条式找配置，并组装紧密数组（每张地图只需做一次，可用于生成一串派生环境场）

In [3]:
import re
from pathlib import Path
import json

"""
数据处理步骤 (Metaphysical Data Processing Steps):

1.  **数据源与目标确立 (Source & Target Definition)**
    - Source: 位于 `precompute/data` 下的静态配置表 (JSON)，定义了游戏世界的逻辑结构。
    - Target: 位于 `ExportedData` 下的 Numpy 数组，代表了具体场景的物理数据 (Voxel/Grid)。

2.  **标识符提取 (Identifier Extraction)**
    - 从物理数据的文件名/路径中提取关键标识符 (SceneID / AssetID)。
    - 例如: 从 `Fishing_1006001_Global.npy` 提取 `1006001`。

3.  **逻辑映射建立 (Logical Mapping)**
    - 利用中间件/配置表 (`map_scene.json`) 将物理标识符 (AssetID) 映射回系统逻辑标识符 (MapID)。
    - 这一步是连接“即时演算数据”与“策划配置数据”的关键桥梁。

4.  **上下文关联与整合 (Context Association & Integration)**
    - 以 MapID 为锚点，级联查询关联的 Pond, Stock, Fish 配置。
    - 将分散的数据整合为适合并行计算的结构化格式 (Numpy/Pandas)。
"""

# 配置路径
DATA_ROOT = Path(r'D:\fishinggame\precompute\data\1\1001')
EXPORTED_DATA_ROOT = Path(r'D:\fishinggame\ExportedData')

# 加载 map_scene.json
with open(DATA_ROOT / 'map_scene.json', 'r', encoding='utf-8') as f:
    map_scene = json.load(f)

# 建立 assetId -> map_id 的反向索引
asset_to_map = {info['assetId']: int(map_id) for map_id, info in map_scene.items() if info.get('assetId')}

In [4]:
def get_scene_id_from_path(npy_path: str) -> str:
    """从npy文件路径中提取scene_id (如 Fishing_1006001_Global.npy -> '1006001')"""
    match = re.search(r'Fishing_(\d+)', str(npy_path))
    if not match:
        raise ValueError(f'无法从路径中提取scene_id: {npy_path}')
    return match.group(1)

def get_map_id_from_scene_id(scene_id: str) -> int:
    """根据scene_id查找对应的map_id"""
    if scene_id in asset_to_map:
        return asset_to_map[scene_id]
    raise ValueError(f'找不到scene_id {scene_id} 对应的map_id')

def get_map_id_from_npy_path(npy_path: str) -> int:
    """从npy文件路径直接获取map_id"""
    scene_id = get_scene_id_from_path(npy_path)
    return get_map_id_from_scene_id(scene_id)

# 测试示例
test_path = r'D:\fishinggame\ExportedData\Fishing_1006001_Dense_20260107_154037\Fishing_1006001_Global.npy'
scene_id = get_scene_id_from_path(test_path)
map_id = get_map_id_from_scene_id(scene_id)
print(f'文件路径: {test_path}')
print(f'提取的 scene_id: {scene_id}')
print(f'对应的 map_id: {map_id}')
print(f'地图信息: {map_scene[str(map_id)]}')

文件路径: D:\fishinggame\ExportedData\Fishing_1006001_Dense_20260107_154037\Fishing_1006001_Global.npy
提取的 scene_id: 1006001
对应的 map_id: 1009
地图信息: {'id': 1009, 'name': 'map_base_6', 'desc': 106, 'assetId': '1006001', 'originOffsetX': 0, 'originOffsetY': 0, 'offsetX': 0, 'offsetY': 0, 'sizeX': 1000, 'sizeY': 1000, 'rotate': 0, 'mark': ''}


#### 对于局部池取stock
* 
* 去D:\fishinggame\precompute\data\1\1001\fish_stock.json当中，

In [5]:
# 加载额外的配置表
with open(DATA_ROOT / 'fish_pond_list.json', 'r', encoding='utf-8') as f:
    fish_pond_list = json.load(f)

with open(DATA_ROOT / 'fish_stock.json', 'r', encoding='utf-8') as f:
    fish_stock_config = json.load(f)

print(f"已加载 {len(fish_pond_list)} 个鱼塘配置")
print(f"已加载 {len(fish_stock_config)} 个 Stock 配置")

# 获取 map_id 对应的 map_scene 配置
current_map_info = map_scene.get(str(map_id))
if not current_map_info:
    raise ValueError(f"Found no map info for id {map_id}")

map_desc_id = current_map_info.get('desc')
print(f"\n当前地图: {current_map_info['name']} (ID: {map_id})")
print(f"关联的 Desc ID (用于对应 fish_pond_list.mapId): {map_desc_id}")

# 查找关联的 Pond 和 Stock
print(f"\n查找 mapId == {map_desc_id} 的鱼塘...")
related_ponds = [pond for pond in fish_pond_list.values() if pond.get('mapId') == map_desc_id]

if not related_ponds:
    print("警告: 未找到关联的鱼塘配置 (Pond)")
else:
    print(f"找到 {len(related_ponds)} 个关联鱼塘:")
    for pond in related_ponds:
        # 兼容处理: 有些json key可能是str类型的id
        pond_id = pond.get('id')
        pond_name = pond.get('name')
        stock_id = pond.get('fishStockId')
        
        print(f"  - Pond: {pond_name} (ID: {pond_id}) -> Stock ID: {stock_id}")
        
        # 查询 Stock 详情 (注意 key 可能是字符串)
        stock_info = fish_stock_config.get(str(stock_id))
        if stock_info:
             print(f"    Stock 详情: Name={stock_info.get('name')}, ResetTime={stock_info.get('resetDayTime')}")
        else:
             print(f"    警告: 在 fish_stock.json 中未找到 Stock ID {stock_id}")

已加载 7 个鱼塘配置
已加载 6 个 Stock 配置

当前地图: map_base_6 (ID: 1009)
关联的 Desc ID (用于对应 fish_pond_list.mapId): 106

查找 mapId == 106 的鱼塘...
找到 1 个关联鱼塘:
  - Pond: Sunset_Stream (ID: 301020005) -> Stock ID: 301030106
    Stock 详情: Name=stock_sunset, ResetTime=05:00


In [4]:
# 顺着往下进行数据查找和组装numpy，供后面的计算使用。
# 大致思路为：
# 5. 遍历 Stock ID (Stock -> Release)：
#    - 从相关联的池塘中提取 Stock ID，查找其下属的所有 Release ID。
# 6. Eager Loading (Release -> Fish/Env):
#    - 对每个 Release ID，立即提取所需的全部配置信息，包括：
#      - 基础属性: qualityId (即原 fishId), weight/length ranges (min/max).
#      - 关键系数: minEnvCoeff, minAdaptCoeff.
#      - 关联元数据: speciesId, envAffinityId (即原 envId).
# 7. 组装 DataFrame (Assembly):
#    - 将上述所有提取的字段扁平化，组装成 Pandas DataFrame (`stockFishesPd`)。
#    - 每一行代表一个 Release 配置，为后续的概率计算和环境场生成做准备。

In [6]:
import pandas as pd

# Load additional configurations
print("Loading Release and Quality configs...")
with open(DATA_ROOT / 'stock_release.json', 'r', encoding='utf-8') as f:
    stock_release_config = json.load(f)

with open(DATA_ROOT / 'fish_release.json', 'r', encoding='utf-8') as f:
    fish_release_config = json.load(f)

with open(DATA_ROOT / 'basic_fish_quality.json', 'r', encoding='utf-8') as f:
    basic_fish_quality_config = json.load(f)
print("Configs loaded.")

rows = []

# 'related_ponds' should be available from the previous cell execution
# If not, we rely on the logic that this cell is run after Cell 5.
if 'related_ponds' not in locals():
    print("Warning: 'related_ponds' not found. Please ensure the previous cell is executed.")
    unique_stock_ids = set()
else:
    unique_stock_ids = set(pond.get('fishStockId') for pond in related_ponds if pond.get('fishStockId'))

print(f"Processing {len(unique_stock_ids)} unique Stock IDs associated with the current map.")

for stock_id in unique_stock_ids:
    # Find all releases for this stock
    # Note: Scanning all values in stock_release_config might be inefficient for very large datasets,
    # but acceptable for this precompute scope.
    stock_releases = [item for item in stock_release_config.values() if item.get('stockId') == stock_id]
    
    for sr in stock_releases:
        release_id = sr.get('releaseId')
        fish_quality_id = sr.get('fishId') # referred as fishId in stock_release.json, but actually qualityId
        fish_env_affinity_id = sr.get('fishEnvId') # referred as fishEnvId in stock_release.json
        
        # Lookup Release Info
        release_info = fish_release_config.get(str(release_id))
        if not release_info:
            # print(f"Warning: Release ID {release_id} not found in fish_release.json")
            continue
            
        # Lookup Fish Quality Info
        fish_info = basic_fish_quality_config.get(str(fish_quality_id))
        species_id = fish_info.get('species', -1) if fish_info else -1
            
        row = {
            'stockId': stock_id,
            'releaseId': release_id,
            'qualityId': fish_quality_id,
            'envAffinityId': fish_env_affinity_id, # Renamed from envId for clarity
            'speciesId': species_id,
            
            # Release Limits
            'weight_min': release_info.get('weightMin'),
            'weight_max': release_info.get('weightMax'),
            'len_min': release_info.get('lengthMin'),
            'len_max': release_info.get('lengthMax'),

            # Environment Coefficients (Added per request)
            'minEnvCoeff': release_info.get('minEnvCoeff', 0),
            'minAdaptCoeff': release_info.get('minAdaptCoeff', 0),
            
            # Debug/Display info
            'name': release_info.get('name'),
            'probWeight': release_info.get('probWeightIdeal')
        }
        rows.append(row)

stockFishesPd = pd.DataFrame(rows)
print(f"Created stockFishesPd with {len(stockFishesPd)} rows.")
if not stockFishesPd.empty:
    print(stockFishesPd.head().to_string())
    print("\nColumn Types:")
    print(stockFishesPd.dtypes)
else:
    print("DataFrame is empty. Check if stock_release.json maps correctly to the pond stock IDs.")

Loading Release and Quality configs...
Configs loaded.
Processing 1 unique Stock IDs associated with the current map.
Created stockFishesPd with 35 rows.
     stockId  releaseId  qualityId  envAffinityId  speciesId  weight_min  weight_max  len_min  len_max  minEnvCoeff  minAdaptCoeff                                name  probWeight
0  301030106     300500  101034430        1013390  101020063         150         450       26       37            0              0  Release_American_Shad_Young_sunset      250000
1  301030106     300510  101034090        1013050  101020010          50         200       16       26            0              0    Release_Brook_Trout_Young_sunset      100000
2  301030106     300520  101031007        1010066  101020010         200         350       26       32            0              0   Release_Brook_Trout_Common_sunset      100000
3  301030106     300530  101034450        1013410  101020003         150         450       28       40            0              0

In [6]:
# 继续进行后续数据关联 (Data Enrichment Phase II)

# 8. 环境亲和性关联 (Environment Affinity Lookup):
#    - 目标: 丰富鱼类的环境适应参数。
#    - 操作: 使用 `envAffinityId` (原 `fishEnvId`) 关联 `fish_env_affinity.json`。
#    - 提取关键属性 (Attributes Extraction):
#         - 基础ID关联: structId (结构), tempId (温度), layerId (水层), lightId (光照)。
#         - 诱鱼系数: baitCoeffGroup, baitTypeCoeffGroup, periodCoeffGroup (时段)。
#         - 适应性参数: 
#             - pressureSensitivity (气压敏感度)
#             - minAdaptLureRatio / maxAdaptLureRatio (路亚适应比例)
#             - maxAcceptLengthRatio (最大接受长度比)
#         - 衰减配置: underLengthDecayCoeff / overLengthDecayCoeff (体型偏离衰减)。

# 9. 结构体亲和性级联查找 (Structure Affinity Cascade):
#    - 目标: 获取具体的物理结构交互参数。
#    - 操作: 使用步骤 8 获得的 `structId`，查询 `struct_affinity.json`。
#    - 提取参数 (Parameters Extraction): 
#         - `List`: 包含 `structType` (结构类型) 和 `coeff` (系数) 的列表。

# 10. 温度亲和性级联查找 (Temperature Affinity Cascade):
#    - 目标: 获取鱼类对温度的敏感度配置。
#    - 操作: 使用步骤 8 获得的 `tempId`，查询 `temp_affinity.json`。
#    - 提取参数 (Parameters Extraction): 
#         - `temperatureFav`: 最适温度 (注意可能需要缩放，如 220 -> 22.0)。
#         - `tempAffectedRatio`: 温度影响比率。
#         - `tempThreshold`: 温度容忍阈值。

# 11. 水层亲和性级联查找 (Water Layer Affinity Cascade):
#    - 目标: 获取鱼类在不同水层的分布偏好。
#    - 操作: 使用步骤 8 获得的 `layerId`，查询 `water_layer_affinity.json`。
#    - 提取参数 (Parameters Extraction): 
#         - `List`: 包含 `layerType` (水层类型, 如上/中/下) 和 `coeff` (系数) 的列表。

In [7]:
# 9-11. 实现环境与亲和性级联查找 (Env & Affinity Cascade Lookup)

print("Loading Affinity Configs...")
# 1. 加载所有亲和性配置 (Load Configs)
with open(DATA_ROOT / 'fish_env_affinity.json', 'r', encoding='utf-8') as f:
    env_affinity_config = json.load(f)

with open(DATA_ROOT / 'struct_affinity.json', 'r', encoding='utf-8') as f:
    struct_affinity_config = json.load(f)

with open(DATA_ROOT / 'temp_affinity.json', 'r', encoding='utf-8') as f:
    temp_affinity_config = json.load(f)

with open(DATA_ROOT / 'water_layer_affinity.json', 'r', encoding='utf-8') as f:
    layer_affinity_config = json.load(f)
print("Affinity Configs loaded.")

# 2. 准备查找字典 (Prepare Lookup Dicts)
#    优化: 直接构建 id -> data 的快速查找字典，避免每次遍历 list
#    注意: JSON key通常是字符串, DataFrame中Id可能是int, 查找时需注意类型转换

def get_config_by_id(config_dict, target_id):
    """Safe lookup helper handling str/int key mismatch"""
    if target_id is None:
        return None
    return config_dict.get(str(target_id))

# 3. 扩展 DataFrame (Enrich DataFrame)
#    虽然可以使用 apply，但在列数较多且逻辑复杂时，迭代或列表推导式便于调试和错误处理
#    考虑到数据量不大 (几十到几百行)，直接遍历 row 更新字典列表然后重新创建 DF 也是一种清晰的方法
#    或者使用 apply + Series expand

def enrich_row(row):
    # Step 8: Env Affinity Lookup
    env_id = row.get('envAffinityId')
    env_info = get_config_by_id(env_affinity_config, env_id)
    
    extra_data = {}
    
    if env_info:
        # Extract basic Env IDs
        struct_id = env_info.get('structId')
        temp_id = env_info.get('tempId')
        layer_id = env_info.get('layerId')
        light_id = env_info.get('lightId')
        
        extra_data.update({
            'structId': struct_id,
            'tempId': temp_id,
            'layerId': layer_id,
            'lightId': light_id,
            # Coeffs
            'baitCoeffGroup': env_info.get('baitCoeffGroup'),
            'baitTypeCoeffGroup': env_info.get('baitTypeCoeffGroup'),
            'periodCoeffGroup': env_info.get('periodCoeffGroup'),
            # Adaptability Stats
            'pressureSensitivity': env_info.get('pressureSensitivity'),
            'minAdaptLureRatio': env_info.get('minAdaptLureRatio'),
            'maxAdaptLureRatio': env_info.get('maxAdaptLureRatio'),
            'maxAcceptLengthRatio': env_info.get('maxAcceptLengthRatio'),
            'underLengthDecayCoeff': env_info.get('underLengthDecayCoeff'),
            'overLengthDecayCoeff': env_info.get('overLengthDecayCoeff'),
        })
        
        # Step 9: Structure Affinity Cascade
        struct_info = get_config_by_id(struct_affinity_config, struct_id)
        if struct_info:
            extra_data['structList'] = struct_info.get('List') # raw list of {structType, coeff}
        
        # Step 10: Temperature Affinity Cascade
        temp_info = get_config_by_id(temp_affinity_config, temp_id)
        if temp_info:
            extra_data['temperatureFav'] = temp_info.get('temperatureFav')
            extra_data['tempAffectedRatio'] = temp_info.get('tempAffectedRatio')
            extra_data['tempThreshold'] = temp_info.get('tempThreshold')
            
        # Step 11: Water Layer Affinity Cascade
        layer_info = get_config_by_id(layer_affinity_config, layer_id)
        if layer_info:
            extra_data['layerList'] = layer_info.get('List') # raw list of {layerType, coeff}
            
    return pd.Series(extra_data)

# 应用扩展逻辑
print("Enriching DataFrame...")
if not stockFishesPd.empty:
    enriched_columns = stockFishesPd.apply(enrich_row, axis=1)
    
    # Concatenate original df with new columns
    stockFishesPd = pd.concat([stockFishesPd, enriched_columns], axis=1)
    
    print("Enrichment Complete.")
    print(f"New DataFrame Shape: {stockFishesPd.shape}")
    print(stockFishesPd[['qualityId','envAffinityId','temperatureFav', 'structId', 'tempId', 'layerList', 'baitCoeffGroup', 'baitTypeCoeffGroup', 'periodCoeffGroup']].head().to_string())
else:
    print("stockFishesPd is empty, skipping enrichment.")

Loading Affinity Configs...
Affinity Configs loaded.
Enriching DataFrame...
Enrichment Complete.
New DataFrame Shape: (35, 31)
   qualityId  envAffinityId  temperatureFav  structId   tempId                                                                                   layerList  baitCoeffGroup  baitTypeCoeffGroup  periodCoeffGroup
0  101034430        1013390             195   2011030  2021010  [{'layerType': 1, 'coeff': 1}, {'layerType': 2, 'coeff': 1}, {'layerType': 3, 'coeff': 1}]           80000                3105              2011
1  101034090        1013050             195   2010940  2020920  [{'layerType': 1, 'coeff': 1}, {'layerType': 2, 'coeff': 1}, {'layerType': 3, 'coeff': 1}]           80000                3096              2011
2  101031007        1010066             195   2010940  2020920  [{'layerType': 1, 'coeff': 1}, {'layerType': 2, 'coeff': 1}, {'layerType': 3, 'coeff': 1}]           80000                3096              2011
3  101034450        1013410          

In [8]:
# 12-14. 诱鱼与时段亲和性关联 (Bait & Period Affinity)

import json
import pandas as pd

print("Loading Bait/Period Configs...")
# Load JSONs
with open(DATA_ROOT / 'bait_affinity.json', 'r', encoding='utf-8') as f:
    bait_affinity_data = json.load(f)
with open(DATA_ROOT / 'bait_type_affinity.json', 'r', encoding='utf-8') as f:
    bait_type_affinity_data = json.load(f)
with open(DATA_ROOT / 'period_affinity.json', 'r', encoding='utf-8') as f:
    period_affinity_data = json.load(f)

# Helper to aggregate by group
# Transform {id: {data}} -> {group_id: [data_list]}
def aggregate_by_group(source_data, group_key_name):
    grouped = {}
    for item in source_data.values():
        grp_id = item.get(group_key_name)
        
        # Safe cast to string for consistent lookup key
        if grp_id is not None:
            grp_key = str(int(grp_id)) # int -> str to match potential int IDs
            if grp_key not in grouped:
                grouped[grp_key] = []
            grouped[grp_key].append(item)
    return grouped

print("Aggregating Groups...")
# Note: 'baitCoeffGroup' matches the field in fish_env_affinity
bait_groups = aggregate_by_group(bait_affinity_data, 'baitCoeffGroup') 
bait_type_groups = aggregate_by_group(bait_type_affinity_data, 'baitTypeCoeffGroup') 

# Note: In period_affinity.json, the key is 'periodGroup', but in fish_env it is 'periodCoeffGroup'
period_groups = aggregate_by_group(period_affinity_data, 'periodGroup')

# Enrich Wrapper
def enrich_bait_period(row):
    # Lookup Bait Group
    # row['baitCoeffGroup'] comes from fish_env_affinity, expected to be int or str
    b_val = row.get('baitCoeffGroup')
    if pd.notnull(b_val):
        b_grp = str(int(b_val))
        bait_list = bait_groups.get(b_grp, [])
    else:
        bait_list = []
    
    # Lookup Bait Type Group
    bt_val = row.get('baitTypeCoeffGroup')
    if pd.notnull(bt_val):
        bt_grp = str(int(bt_val))
        bait_type_list = bait_type_groups.get(bt_grp, [])
    else:
        bait_type_list = []
    
    # Lookup Period Group
    p_val = row.get('periodCoeffGroup')
    if pd.notnull(p_val):
        p_grp = str(int(p_val))
        period_list = period_groups.get(p_grp, [])
    else:
        period_list = []
    
    return pd.Series({
        'baitList': bait_list,          # detailed list of {baitId, coeff...}
        'baitTypeList': bait_type_list, # detailed list of {baitSubType, coeff...}
        'periodList': period_list       # detailed list of {periodId, activityFactor...}
    })

print("Enriching StockFishesPd with Bait/Period lists...")
if not stockFishesPd.empty:
    bp_columns = stockFishesPd.apply(enrich_bait_period, axis=1)
    
    # Concatenate
    # Drop existing if re-running to avoid dupe columns
    cols_to_drop = [c for c in ['baitList', 'baitTypeList', 'periodList'] if c in stockFishesPd.columns]
    if cols_to_drop:
         stockFishesPd = stockFishesPd.drop(columns=cols_to_drop)
            
    stockFishesPd = pd.concat([stockFishesPd, bp_columns], axis=1)
    
    print("Enrichment Complete.")
    print(f"DataFrame Shape: {stockFishesPd.shape}")
    
    # Sample Output
    # Only show limited info
    cols_check = ['qualityId', 'baitCoeffGroup', 'baitList', 'periodList']
    # Just print the first row nicely formatted
    first_row = stockFishesPd.iloc[0]
    print(f"Sample Row 0 - QualityID: {first_row['qualityId']}")
    print(f"BaitCoeffGroup: {first_row['baitCoeffGroup']}")
    print(f"BaitList Count: {len(first_row['baitList'])}")
    print(f"PeriodList Count: {len(first_row['periodList'])}")
else:
    print("DataFrame empty.")

Loading Bait/Period Configs...
Aggregating Groups...
Enriching StockFishesPd with Bait/Period lists...
Enrichment Complete.
DataFrame Shape: (35, 34)
Sample Row 0 - QualityID: 101034430
BaitCoeffGroup: 80000
BaitList Count: 1
PeriodList Count: 8


### 核心计算准备 1：数据矩阵化 (Matrix Conversion)
将 DataFrame 中的嵌套 List 转换为稠密 Numpy 矩阵，以便于并行查找和计算。

*   **StructAffinityMatrix**: `[NumFishes, MaxStructTypes]`
*   **LayerAffinityMatrix**: `[NumFishes, MaxLayerTypes]`
*   **TempAffinityParams**: `[NumFishes, 3]` (Fav, Ratio, Threshold)

In [10]:
import numpy as np

# 1. 定义常量
# 假设系统中的结构类型和水层类型最大数量 (根据策划文档或数据扫描确定)
MAX_STRUCT_TYPES = 20  # structId 范围通常在一定区间，这里假设映射到0-19或直接用TypeID索引，需确认TypeID最大值。
# 实际上 StructType 是枚举值，建议先扫描一遍所有出现的 StructType 建立映射，或者直接用 TypeID 作为索引 (如果 ID 较小)。
# 观察数据 structList: [{'structType': 1, 'coeff': 100}, ...]
# 假设 structType 是从 1 开始的小整数。
MAX_STRUCT_TYPE_ID = 30 # 安全起见给大一点

MAX_LAYER_TYPES = 5 # 上中下底等，通常<5


def build_dense_matrices(df):
    num_fishes = len(df)
    
    # --- A. Struct Affinity Matrix ---
    # Shape: [F, S], Init with 0.0 (Assuming 0 affinity if not listed)
    struct_matrix = np.zeros((num_fishes, MAX_STRUCT_TYPE_ID), dtype=np.float16)
    
    # --- B. Layer Affinity Matrix ---
    # Shape: [F, L]
    layer_matrix = np.zeros((num_fishes, MAX_LAYER_TYPES), dtype=np.float16)
    
    # --- C. Temp Affinity Params ---
    # [Fav, Ratio, Threshold]
    temp_params = np.zeros((num_fishes, 3), dtype=np.float16)
    
    # Fill Data
    for idx, row in df.iterrows():
        # 1. Struct
        # row['structList'] is list of dict or NaN
        s_list = row.get('structList')
        if isinstance(s_list, list):
            for item in s_list:
                s_type = item.get('structType')
                coeff = item.get('coeff')
                if s_type is not None and s_type < MAX_STRUCT_TYPE_ID:
                    struct_matrix[idx, s_type] = coeff
        else:
            # If no struct preference, it might mean insensitive (100) or avoid (0).
            # Assuming insensitive (1.0 = 100) for now if not specified.
            struct_matrix[idx, :] = 100 

        # 2. Layer
        l_list = row.get('layerList')
        if isinstance(l_list, list):
            for item in l_list:
                l_type = item.get('layerType')
                coeff = item.get('coeff')
                if l_type is not None and l_type < MAX_LAYER_TYPES:
                    layer_matrix[idx, l_type] = coeff
        else:
             # Default 1.0 if no layer pref? 
             layer_matrix[idx, :] = 1.0
        
        # 3. Temp
        # Params: temperatureFav, tempAffectedRatio, tempThreshold
        t_fav = row.get('temperatureFav')
        t_ratio = row.get('tempAffectedRatio')
        t_thres = row.get('tempThreshold')
        
        if pd.notnull(t_fav):
             temp_params[idx, 0] = t_fav / 10.0 # 220 -> 22.0
        else:
             # Error Out as per decision
             # raise ValueError(f"Missing temperatureFav for fish index {idx}")
             pass
             
        if pd.notnull(t_ratio):
             temp_params[idx, 1] = t_ratio / 10000.0
             
        if pd.notnull(t_thres):
             temp_params[idx, 2] = t_thres / 10000.0

    return struct_matrix, layer_matrix, temp_params

print("Converting to Dense Matrices...")
# Ensure strict ordering by resetting index
stockFishesPd.reset_index(drop=True, inplace=True)
m_struct, m_layer, m_temp = build_dense_matrices(stockFishesPd)

print(f"Struct Matrix: {m_struct.shape}, dtype={m_struct.dtype}")
print(f"Layer Matrix:  {m_layer.shape}, dtype={m_layer.dtype}")
print(f"Temp Params:   {m_temp.shape}, dtype={m_temp.dtype}")

print(f"Max Struct Val: {np.max(m_struct)}")
print(f"Max Layer Val: {np.max(m_layer)}")

### 核心计算准备 2：加载体素数据 (Load Voxel Data)
读取 `Global.npy`，提取 Structural Slots 和 Depth Layer 信息。

*   **Data**: `[X, Z, C]`
*   **Target**: `struct_slots [X, Z, 3]`, `depth_layer [X, Z]` (+ Y extension)

In [11]:
# Load Voxel Data
# test_path comes from previous cells: D:\fishinggame\ExportedData\...\Fishing_1006001_Global.npy
print(f"Loading Voxel Data from: {test_path}")
voxel_data = np.load(test_path)
print(f"Loaded Voxel Data Shape: {voxel_data.shape}, dtype={voxel_data.dtype}")

# Parse based on VoxelMapDataFormat.md (Simplified Assumptions for Demo)
# Assuming:
# Channel 0: Struct Layer 1
# Channel 1: Struct Layer 2
# Channel 2: Struct Layer 3
# Channel 3: Depth/Terrain info ...
# We need to check the mask definition in C# or MD.
# For this demo, let's assume Channels 0,1,2 contain StructTypeIds (or -1).
# Note: In .npy from Unity, usually it's [X, Z, Channels]

# Demo: Slice struct slots
struct_slots_map = voxel_data[:, :, 0:3].astype(np.int32)
# If data is float or other bits, might need decoding. Assuming direct ID for now.

# Demo: Slice depth info (Assuming Channel 3 is Depth/Layer)
# In actual format, might be a bitmask. Let's assume it's normalized depth 0-1 or layer index.
depth_map_raw = voxel_data[:, :, 3]

print(f"Struct Slots Map Shape: {struct_slots_map.shape}")
print(f"Depth Raw Map Shape: {depth_map_raw.shape}")

### 核心计算 3：批量亲和度计算 (Data-Driven Batch Calculation)

1.  **AffStruct**: `StructMatrix` vs `StructSlotsMap` (Gather)
2.  **AffTemp**: `TempParams` vs `DepthMap` (Gaussian)
3.  **Synthesis**: `EnvCoeff`

In [12]:
# -----------------------------------------------------
# A. Struct Affinity Calculation (Simplified 2D for Demo)
# -----------------------------------------------------
# Goal: affStruct[x, z, f] = Max over 3 slots (StructMatrix[f, SlotId])

# Input: 
#   struct_slots_map: [X, Z, 3]
#   m_struct: [F, S]

# Dimensions
X, Z, _ = struct_slots_map.shape
F = m_struct.shape[0]

print(f"Calculating Struct Affinity for Grid [{X}x{Z}] and {F} Fish...")

# Naive Broadcasting Approach (Memory Heavy for huge maps, ok for Demo)
# Advanced: Use Numba for loop-fusion.

# 1. Flatten slots to facilitate indexing
# [X*Z, 3]
slots_flat = struct_slots_map.reshape(-1, 3)

# 2. Gather values from m_struct
# We want result: [X*Z, 3, F]
# StructMatrix is [F, S]. 
# We effectively want m_struct[:, slots_flat]
# But advanced indexing is tricky with mixed dims.

# Let's flip m_struct to [S, F] for easier gathering?
m_struct_T = m_struct.T # [S, F]

# Safe Indexing: Clip negative indices (-1) to 0 or handle them.
# Assume 0 is "No Struct" and has affinity 1.0 or 0.0.
# Let's clip to 0.
slots_safe = np.maximum(slots_flat, 0)

# Gather: [X*Z*3, F] -> Reshape [X*Z, 3, F]
# result = m_struct_T[slots_safe.flatten()] 
# shape: [X*Z*3, F]
aff_struct_gathered = m_struct_T[slots_safe.flatten()].reshape(X, Z, 3, F)

# 3. Max over slots -> [X, Z, F]
aff_struct_final = np.max(aff_struct_gathered, axis=2)

# Normalize if data was 100-based
aff_struct_final = aff_struct_final / 100.0

print(f"AffStruct Final Shape: {aff_struct_final.shape}")
print("Sample AffStruct (Fish 0, Grid 0:5,0:5):")
print(aff_struct_final[0:5, 0:5, 0])


# -----------------------------------------------------
# B. Temp Affinity Calculation (Depth-based)
# -----------------------------------------------------
# Goal: affTemp[y, f] (Simplified to [y] in demo, actually depth varies)
# For Demo, let's assume DepthMap implies a T_val per pixel.
# T_pixel = SurfaceT + (BottomT - SurfaceT) * DepthMap (0-1)

SURFACE_T = 25.0
BOTTOM_T = 10.0

# Temp Map [X, Z]
t_map = SURFACE_T + (BOTTOM_T - SURFACE_T) * (depth_map_raw / 255.0) # Assuming 8-bit depth

# Calc Affinity: exp( - (T - Tfav)^2 / (Width * Ratio^2) )
# m_temp: [F, 3] -> Fav, Ratio, Thres

# Broadcast T_map to [X, Z, F]
t_map_expanded = t_map[:, :, np.newaxis] # [X, Z, 1]

t_fav = m_temp[:, 0] # [F]
t_ratio = m_temp[:, 1] # [F]
# Width constant
WIDTH_CONST = 50.0 # From guide/config

# (T - Tfav)^2
diff_sq = (t_map_expanded - t_fav) ** 2 # [X, Z, F]

# Denom: Width * Ratio^2
# Avoid div by zero
denom = WIDTH_CONST * (t_ratio ** 2)
denom[denom < 1e-5] = 1e-5

aff_temp_final = np.exp(- diff_sq / denom)

print(f"AffTemp Final Shape: {aff_temp_final.shape}")
print("Sample AffTemp (Fish 0, Grid 0:5,0:5):")
print(aff_temp_final[0:5, 0:5, 0])

# -----------------------------------------------------
# C. Synthesis
# -----------------------------------------------------
# EnvCoeff = AffStruct * AffTemp * ...

env_coeff_final = aff_struct_final * aff_temp_final

print(f"EnvCoeff Final Shape: {env_coeff_final.shape}")
print("Sample EnvCoeff (Fish 0, Grid 0:5,0:5):")
print(env_coeff_final[0:5, 0:5, 0])